In [10]:
# import
import pandas as pd
# this gets rid of some random warning when separating data into EPC1 & EPC2
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np

input1 = 'Data/REV3 3m/6 Lat to Front/6_2s_2_2024-06-19_14-45-00.csv'
input2 = 'Data/REV3 3m/6 Lat to Front/6_2s_2_2024-06-19_14-45-14.csv'
input3 = 'Data/REV3 3m/6 Lat to Front/6_2s_2_2024-06-19_14-45-28.csv'
inputs = [input1, input2, input3]

# other variables
# do i want this?
phase = 'PhaseAngle'
RSSI = 'RSSI'
title = '1: Lateral Raise Gesture - REV3 3m Data'

# load csv file into dataframe and change header row (deleted everything before)
data = []
for input in inputs:
    data.append(pd.read_csv(input, header = 2))
#data = pd.read_csv(input, header = 2)

for i in range(len(data)):
    # change dataframe headers to correct column and select wanted data
    data[i] = data[i].iloc[0:, [0,1,4,7]]

    # rename columns
    data[i].rename(columns={'// Timestamp' : 'Timestamp'},     inplace=True)
    data[i].rename(columns={' RSSI'        : 'RSSI'},          inplace=True)
    data[i].rename(columns={' PhaseAngle'  : 'PhaseAngle'},    inplace = True)
    data[i].rename(columns={' EPC'         : 'EPC'},           inplace=True)

    # parse timestamp data and create new column to have comparable time values
    data[i]['Timestamp'] = pd.to_datetime(data[i]['Timestamp'])
    first_timestamp = data[i]['Timestamp'].iloc[0]
    data[i]['TimeElapsed'] = data[i]['Timestamp'] - first_timestamp

    # separate time elapsed into seconds and milliseconds, then combine the numbers
    data[i]['seconds'] = data[i]['TimeElapsed'].dt.seconds
    data[i]['milliseconds'] = data[i]['TimeElapsed'].dt.microseconds // 1000
    data[i]['TimeValue'] = data[i]['seconds'] + data[i]['milliseconds'] / 1000
    data[i]['TimeValue'] = data[i]['TimeValue'].round(4) # round to 4 decimal places

    # move timevalue column over
    data[i]['Timestamp'] = data[i]['TimeValue']
    data[i].rename(columns={'Timestamp': 'TimeValue'}, inplace=True)

    # remove excess time columns, good place to print data
    data[i] = data[i].iloc[0:, [0,1,2,3]]

    # correctly change RSSI column to numbers
    if(data[i][RSSI].dtype != 'int64'):
        # replace commas, negative signs, then change to float64s
        data[i][RSSI] = data[i][RSSI].str.replace(',' , '.', regex = False)
        data[i][RSSI] = data[i][RSSI].str.replace(r'[^\d.-]', '-', regex = True)
        data[i][RSSI] = pd.to_numeric(data[i][RSSI], errors='coerce')

    # replace commas, change to float64s, and round
    data[i][phase] = data[i][phase].str.replace(',' , '.', regex = False)
    data[i][phase] = pd.to_numeric(data[i][phase])
    data[i][phase] = data[i][phase].round(4)

    mapping = {'A10000000000000000000000': 1, 'A20000000000000000000000': 2}
    # Replace the strings with numbers
    data[i]['EPC'] = data[i]['EPC'].replace(mapping)

    